# Praktikum Minggu 7: Penyimpanan Big Data
## *Week 7 Lab: Big Data Storage – NoSQL, Cloud Storage & Data Lake*

**Mata Kuliah:** Big Data Analytics  
**Topik:** NoSQL Simulation, Cloud Storage Concepts, SQLite, BigQuery, Data Lake

---
### Tujuan Praktikum
1. Mensimulasikan operasi NoSQL Document Store dan Key-Value Store
2. Memahami perbedaan format file (CSV vs Parquet)
3. Menggunakan SQLite untuk analisis data lokal
4. Mengenal query BigQuery pada public dataset
5. Mensimulasikan arsitektur Data Lake (Bronze/Silver/Gold)

In [ ]:
# Install library tambahan
!pip install google-cloud-bigquery google-cloud-storage db-dtypes pyarrow fastparquet --quiet
print('✅ Instalasi selesai')

## 1. Simulasi NoSQL: Document Store dengan Python Dict

MongoDB menyimpan data sebagai dokumen JSON. Kita akan mensimulasikan operasi CRUD (Create, Read, Update, Delete) menggunakan dictionary Python.

In [ ]:
import json
import copy
from datetime import datetime

# ============================================================
# Simulasi Document Store (seperti MongoDB)
# ============================================================

class DocumentStore:
    """Simulasi sederhana MongoDB-like document store."""

    def __init__(self, name):
        self.name = name
        self._store = {}  # {id: document}
        self._counter = 1

    def insert_one(self, document):
        """INSERT: Menyimpan dokumen baru."""
        doc = copy.deepcopy(document)
        doc_id = f"id_{self._counter:04d}"
        doc['_id'] = doc_id
        doc['_created_at'] = datetime.now().isoformat()
        self._store[doc_id] = doc
        self._counter += 1
        return doc_id

    def insert_many(self, documents):
        """INSERT MANY: Menyimpan banyak dokumen sekaligus."""
        ids = [self.insert_one(doc) for doc in documents]
        return ids

    def find_all(self):
        """FIND ALL: Mengembalikan semua dokumen."""
        return list(self._store.values())

    def find_one(self, doc_id):
        """FIND ONE: Mencari dokumen berdasarkan ID."""
        return self._store.get(doc_id, None)

    def find_by(self, field, value):
        """FIND BY FIELD: Query sederhana."""
        return [doc for doc in self._store.values() if doc.get(field) == value]

    def update_one(self, doc_id, updates):
        """UPDATE: Memperbarui field tertentu."""
        if doc_id in self._store:
            self._store[doc_id].update(updates)
            self._store[doc_id]['_updated_at'] = datetime.now().isoformat()
            return True
        return False

    def delete_one(self, doc_id):
        """DELETE: Menghapus satu dokumen."""
        return self._store.pop(doc_id, None) is not None

    def count(self):
        return len(self._store)


# Buat collection 'mahasiswa'
db_mahasiswa = DocumentStore('mahasiswa')

# CREATE - Insert banyak dokumen (perhatikan skema fleksibel!)
mahasiswa_data = [
    {'nama': 'Budi Santoso', 'nim': '2021001', 'prodi': 'Informatika', 'ipk': 3.75,
     'alamat': {'kota': 'Jakarta', 'provinsi': 'DKI Jakarta'},
     'mata_kuliah': ['Big Data', 'Machine Learning', 'Statistika']},
    {'nama': 'Sari Dewi', 'nim': '2021002', 'prodi': 'Sistem Informasi', 'ipk': 3.91,
     'alamat': {'kota': 'Bandung', 'provinsi': 'Jawa Barat'},
     'mata_kuliah': ['Big Data', 'Database', 'Jaringan'],
     'beasiswa': 'BidikMisi'},  # kolom ekstra! Schema-less
    {'nama': 'Andi Wijaya', 'nim': '2021003', 'prodi': 'Informatika', 'ipk': 3.55,
     'alamat': {'kota': 'Surabaya', 'provinsi': 'Jawa Timur'},
     'mata_kuliah': ['Big Data', 'Algoritma']},
    {'nama': 'Maya Putri', 'nim': '2021004', 'prodi': 'Data Science', 'ipk': 3.88,
     'alamat': {'kota': 'Yogyakarta', 'provinsi': 'DIY'},
     'mata_kuliah': ['Big Data', 'Deep Learning', 'Statistika', 'Visualisasi'],
     'publikasi': 2},  # kolom ekstra!
]

ids = db_mahasiswa.insert_many(mahasiswa_data)
print(f'✅ INSERT: {len(ids)} dokumen berhasil disimpan. IDs: {ids}')

# READ - Baca semua
print(f'\n📖 Jumlah dokumen dalam collection: {db_mahasiswa.count()}')

# READ - Baca satu
doc = db_mahasiswa.find_one('id_0001')
print(f'\n📄 Dokumen id_0001:')
print(json.dumps(doc, indent=2, ensure_ascii=False))

# QUERY - Cari berdasarkan prodi
informatika = db_mahasiswa.find_by('prodi', 'Informatika')
print(f'\n🔍 Query: mahasiswa prodi Informatika = {len(informatika)} orang')
for m in informatika:
    print(f"   - {m['nama']} (IPK: {m['ipk']})")

# UPDATE
updated = db_mahasiswa.update_one('id_0001', {'ipk': 3.82, 'semester': 6})
print(f'\n✏️  UPDATE id_0001 berhasil: {updated}')
print(f"   IPK baru: {db_mahasiswa.find_one('id_0001')['ipk']}")

# DELETE
deleted = db_mahasiswa.delete_one('id_0003')
print(f'\n🗑️  DELETE id_0003 berhasil: {deleted}')
print(f'   Sisa dokumen: {db_mahasiswa.count()}')

print('\n✅ Simulasi Document Store selesai!')
print('   Keunggulan utama: SKEMA FLEKSIBEL - tiap dokumen bisa memiliki field berbeda')

## 2. Simulasi Key-Value Store

Redis adalah in-memory key-value store yang sangat cepat. Kita akan mensimulasikan operasi dasarnya, termasuk konsep TTL (Time-To-Live) untuk expiration.

In [ ]:
import time

# ============================================================
# Simulasi Key-Value Store (seperti Redis)
# ============================================================

class KeyValueStore:
    """Simulasi Redis-like in-memory key-value store."""

    def __init__(self):
        self._data = {}         # {key: value}
        self._expiry = {}       # {key: expire_timestamp}

    def _is_expired(self, key):
        if key in self._expiry:
            if time.time() > self._expiry[key]:
                del self._data[key]
                del self._expiry[key]
                return True
        return False

    def set(self, key, value, ex=None):
        """SET key value [EX seconds]: Simpan nilai."""
        self._data[key] = value
        if ex:
            self._expiry[key] = time.time() + ex
        return 'OK'

    def get(self, key):
        """GET key: Ambil nilai berdasarkan kunci."""
        if self._is_expired(key):
            return None
        return self._data.get(key, None)

    def delete(self, *keys):
        """DEL key [key ...]: Hapus satu atau lebih kunci."""
        count = 0
        for key in keys:
            if key in self._data:
                del self._data[key]
                self._expiry.pop(key, None)
                count += 1
        return count

    def exists(self, key):
        """EXISTS key: Cek apakah kunci ada."""
        return not self._is_expired(key) and key in self._data

    def keys(self, pattern='*'):
        """KEYS: Tampilkan semua kunci."""
        import fnmatch
        return [k for k in self._data if not self._is_expired(k) and
                (pattern == '*' or fnmatch.fnmatch(k, pattern))]

    def ttl(self, key):
        """TTL key: Sisa waktu hidup."""
        if key not in self._expiry:
            return -1  # tidak expire
        remaining = self._expiry[key] - time.time()
        return max(0, remaining)

    def incr(self, key):
        """INCR key: Increment nilai integer."""
        val = int(self._data.get(key, 0)) + 1
        self._data[key] = str(val)
        return val

    def hset(self, name, field, value):
        """HSET: Simpan hash (nested dict)."""
        if name not in self._data:
            self._data[name] = {}
        self._data[name][field] = value

    def hget(self, name, field):
        """HGET: Ambil field dari hash."""
        return self._data.get(name, {}).get(field, None)

    def hgetall(self, name):
        """HGETALL: Ambil semua field dari hash."""
        return self._data.get(name, {})


# ============================================================
# Demo Operasi Key-Value Store
# ============================================================
cache = KeyValueStore()

print('=== Simulasi Key-Value Store (Redis-like) ===')

# SET / GET
print('\n[1] SET & GET operasi dasar')
cache.set('nama_aplikasi', 'BigDataApp v2.0')
cache.set('max_connections', '100')
cache.set('environment', 'production')
print(f"  GET nama_aplikasi → {cache.get('nama_aplikasi')}")
print(f"  GET max_connections → {cache.get('max_connections')}")

# Session management dengan TTL
print('\n[2] Session Management dengan TTL (expiry)')
cache.set('session:user_budi', 'token_abc123xyz', ex=2)  # expire dalam 2 detik
print(f"  GET session:user_budi (sebelum expire) → {cache.get('session:user_budi')}")
print(f"  TTL session:user_budi → {cache.ttl('session:user_budi'):.2f} detik")
time.sleep(2.1)
print(f"  GET session:user_budi (setelah 2.1 detik) → {cache.get('session:user_budi')} (None = expired)")

# Counter
print('\n[3] Counter (INCR)')
for _ in range(5):
    val = cache.incr('halaman_views')
print(f"  Halaman views counter: {cache.get('halaman_views')}")

# Hash - menyimpan objek
print('\n[4] Hash - Menyimpan objek pengguna')
cache.hset('user:1001', 'nama', 'Sari Dewi')
cache.hset('user:1001', 'email', 'sari@email.com')
cache.hset('user:1001', 'role', 'admin')
print(f"  HGETALL user:1001 → {cache.hgetall('user:1001')}")
print(f"  HGET user:1001 email → {cache.hget('user:1001', 'email')}")

# Keys pattern
print('\n[5] Semua kunci yang ada:')
print(f"  KEYS * → {cache.keys()}")

# DELETE
deleted = cache.delete('max_connections', 'environment')
print(f'\n[6] DELETE → {deleted} kunci dihapus')
print(f"  KEYS * (setelah delete) → {cache.keys()}")

print('\n✅ Simulasi Key-Value Store selesai!')

## 3. Bekerja dengan File di Google Colab (Simulasi Cloud Storage)

Di cloud (AWS S3, GCS), file disimpan sebagai objek. Kita akan mensimulasikan ini dengan membandingkan format **CSV vs Parquet** dalam hal ukuran file dan kecepatan baca.

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import tempfile

# Buat dataset simulasi besar
print('Membuat dataset simulasi...')
np.random.seed(42)
n = 100_000

df_sim = pd.DataFrame({
    'id': range(1, n + 1),
    'timestamp': pd.date_range('2024-01-01', periods=n, freq='min'),
    'user_id': np.random.randint(1000, 9999, n),
    'product_id': np.random.choice(['P001', 'P002', 'P003', 'P004', 'P005'], n),
    'category': np.random.choice(['Elektronik', 'Pakaian', 'Makanan', 'Buku', 'Olahraga'], n),
    'amount': np.round(np.random.exponential(150_000, n), 2),
    'quantity': np.random.randint(1, 10, n),
    'discount_pct': np.round(np.random.uniform(0, 30, n), 1),
    'is_paid': np.random.choice([True, False], n, p=[0.85, 0.15]),
    'region': np.random.choice(['Jawa', 'Sumatera', 'Kalimantan', 'Sulawesi', 'Papua'], n),
})
print(f'Dataset: {n:,} baris x {len(df_sim.columns)} kolom')

# Simpan ke tempdir
tmpdir = tempfile.mkdtemp()
csv_path = os.path.join(tmpdir, 'transaksi.csv')
parquet_path = os.path.join(tmpdir, 'transaksi.parquet')
parquet_compressed_path = os.path.join(tmpdir, 'transaksi_snappy.parquet')

# Tulis CSV
t0 = time.time()
df_sim.to_csv(csv_path, index=False)
csv_write_time = time.time() - t0

# Tulis Parquet (uncompressed)
t0 = time.time()
df_sim.to_parquet(parquet_path, index=False, compression=None)
parquet_write_time = time.time() - t0

# Tulis Parquet (Snappy compression)
t0 = time.time()
df_sim.to_parquet(parquet_compressed_path, index=False, compression='snappy')
parquet_snappy_write_time = time.time() - t0

# Baca kembali
t0 = time.time()
_ = pd.read_csv(csv_path)
csv_read_time = time.time() - t0

t0 = time.time()
_ = pd.read_parquet(parquet_path)
parquet_read_time = time.time() - t0

t0 = time.time()
_ = pd.read_parquet(parquet_compressed_path)
parquet_snappy_read_time = time.time() - t0

# Bandingkan
csv_size = os.path.getsize(csv_path)
parquet_size = os.path.getsize(parquet_path)
parquet_snappy_size = os.path.getsize(parquet_compressed_path)

print('\n' + '=' * 65)
print(f'{"FORMAT":<25} {"UKURAN":>12} {"TULIS(s)":>10} {"BACA(s)":>10}')
print('=' * 65)
print(f'{"CSV":<25} {csv_size/1024/1024:>10.2f}MB {csv_write_time:>10.3f}s {csv_read_time:>10.3f}s')
print(f'{"Parquet (uncompressed)":<25} {parquet_size/1024/1024:>10.2f}MB {parquet_write_time:>10.3f}s {parquet_read_time:>10.3f}s')
print(f'{"Parquet (Snappy)":<25} {parquet_snappy_size/1024/1024:>10.2f}MB {parquet_snappy_write_time:>10.3f}s {parquet_snappy_read_time:>10.3f}s')
print('=' * 65)
print(f'Penghematan ukuran Parquet Snappy vs CSV: {(1 - parquet_snappy_size/csv_size)*100:.1f}%')

# Demo column pruning (keunggulan Parquet)
print('\n=== Demo Columnar Pruning (baca kolom tertentu saja) ===')
t0 = time.time()
_ = pd.read_csv(csv_path, usecols=['user_id', 'amount', 'category'])
csv_col_time = time.time() - t0

t0 = time.time()
_ = pd.read_parquet(parquet_compressed_path, columns=['user_id', 'amount', 'category'])
parquet_col_time = time.time() - t0

print(f'CSV (3 kolom)     : {csv_col_time:.3f}s')
print(f'Parquet (3 kolom) : {parquet_col_time:.3f}s')
print(f'Parquet {(csv_col_time/parquet_col_time):.1f}x lebih cepat untuk column-selective read')

## 4. Pengenalan SQLite sebagai Alternatif Lokal

SQLite adalah database SQL ringan yang tidak memerlukan server. Cocok untuk prototyping dan simulasi analitik data warehouse lokal.

In [ ]:
import sqlite3
import pandas as pd
import os
import tempfile

# Buat database SQLite
db_path = os.path.join(tempfile.mkdtemp(), 'toko_online.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print('=== Membuat Skema Database ===')

# Buat tabel
cursor.executescript("""
    CREATE TABLE IF NOT EXISTS produk (
        id_produk   TEXT PRIMARY KEY,
        nama        TEXT NOT NULL,
        kategori    TEXT,
        harga       REAL,
        stok        INTEGER DEFAULT 0
    );

    CREATE TABLE IF NOT EXISTS pelanggan (
        id_pelanggan INTEGER PRIMARY KEY AUTOINCREMENT,
        nama         TEXT NOT NULL,
        kota         TEXT,
        email        TEXT UNIQUE
    );

    CREATE TABLE IF NOT EXISTS transaksi (
        id_transaksi  INTEGER PRIMARY KEY AUTOINCREMENT,
        id_pelanggan  INTEGER,
        id_produk     TEXT,
        jumlah        INTEGER,
        total_harga   REAL,
        tanggal       DATE,
        FOREIGN KEY (id_pelanggan) REFERENCES pelanggan(id_pelanggan),
        FOREIGN KEY (id_produk) REFERENCES produk(id_produk)
    );
""")
conn.commit()
print('✅ Tabel produk, pelanggan, transaksi berhasil dibuat')

# Insert data
produk_data = [
    ('P001', 'Laptop Asus', 'Elektronik', 8500000, 15),
    ('P002', 'Mouse Wireless', 'Elektronik', 250000, 50),
    ('P003', 'Buku Python DS', 'Buku', 150000, 30),
    ('P004', 'Kaos Polos', 'Pakaian', 75000, 100),
    ('P005', 'Headphone BT', 'Elektronik', 450000, 25),
]
cursor.executemany('INSERT OR IGNORE INTO produk VALUES (?,?,?,?,?)', produk_data)

pelanggan_data = [
    ('Budi Santoso', 'Jakarta', 'budi@email.com'),
    ('Sari Dewi', 'Bandung', 'sari@email.com'),
    ('Andi Wijaya', 'Surabaya', 'andi@email.com'),
    ('Maya Putri', 'Yogyakarta', 'maya@email.com'),
    ('Rizky Fajar', 'Medan', 'rizky@email.com'),
]
cursor.executemany('INSERT OR IGNORE INTO pelanggan (nama, kota, email) VALUES (?,?,?)', pelanggan_data)

transaksi_data = [
    (1, 'P001', 1, 8500000, '2024-01-15'), (2, 'P002', 2, 500000, '2024-01-15'),
    (3, 'P003', 3, 450000, '2024-01-16'), (1, 'P005', 1, 450000, '2024-01-17'),
    (4, 'P004', 5, 375000, '2024-01-18'), (2, 'P001', 1, 8500000, '2024-01-18'),
    (5, 'P002', 3, 750000, '2024-01-19'), (3, 'P005', 2, 900000, '2024-01-20'),
    (4, 'P001', 1, 8500000, '2024-01-20'), (1, 'P003', 5, 750000, '2024-01-21'),
]
cursor.executemany('INSERT INTO transaksi (id_pelanggan, id_produk, jumlah, total_harga, tanggal) VALUES (?,?,?,?,?)', transaksi_data)
conn.commit()
print(f'✅ Data berhasil dimasukkan')

# ============================================================
# Query Analitik SQL
# ============================================================
print('\n=== Query Analitik SQL ===')

queries = {
    'Total Revenue per Kategori': """
        SELECT p.kategori, 
               COUNT(t.id_transaksi) AS jumlah_transaksi,
               SUM(t.total_harga) AS total_revenue,
               AVG(t.total_harga) AS avg_order_value
        FROM transaksi t
        JOIN produk p ON t.id_produk = p.id_produk
        GROUP BY p.kategori
        ORDER BY total_revenue DESC
    """,
    'Top Pelanggan berdasarkan Total Belanja': """
        SELECT pl.nama, pl.kota,
               COUNT(t.id_transaksi) AS jumlah_beli,
               SUM(t.total_harga) AS total_belanja
        FROM transaksi t
        JOIN pelanggan pl ON t.id_pelanggan = pl.id_pelanggan
        GROUP BY pl.id_pelanggan
        ORDER BY total_belanja DESC
        LIMIT 3
    """,
    'Revenue Harian': """
        SELECT tanggal,
               COUNT(*) AS jumlah_transaksi,
               SUM(total_harga) AS daily_revenue
        FROM transaksi
        GROUP BY tanggal
        ORDER BY tanggal
    """
}

for title, query in queries.items():
    print(f'\n--- {title} ---')
    result = pd.read_sql_query(query, conn)
    print(result.to_string(index=False))

conn.close()
print('\n✅ Koneksi SQLite ditutup')

## 5. BigQuery Public Dataset - Contoh Query

Google BigQuery menyediakan banyak **public dataset** yang bisa diquery langsung. Berikut adalah contoh query SQL yang bisa dijalankan di BigQuery Console (https://console.cloud.google.com/bigquery).

In [ ]:
# ============================================================
# Contoh Query BigQuery pada Public Datasets
# CATATAN: Query ini ditampilkan sebagai string/referensi.
# Untuk menjalankannya, Anda perlu autentikasi Google Cloud.
# Bisa langsung dijalankan di: https://console.cloud.google.com/bigquery
# ============================================================

bigquery_queries = {
    'Query 1: Jumlah Kelahiran per Tahun (USA)': {
        'dataset': 'bigquery-public-data.samples.natality',
        'sql': """
SELECT
  year,
  COUNT(*) AS total_births,
  AVG(weight_pounds) AS avg_birth_weight_lbs
FROM `bigquery-public-data.samples.natality`
WHERE year BETWEEN 2000 AND 2008
GROUP BY year
ORDER BY year;
""",
        'penjelasan': 'Menghitung total kelahiran dan rata-rata berat bayi per tahun'
    },
    'Query 2: Top 10 GitHub Repo Paling Aktif': {
        'dataset': 'bigquery-public-data.github_repos.commits',
        'sql': """
SELECT
  repo_name,
  COUNT(*) AS commit_count,
  COUNT(DISTINCT author.email) AS unique_contributors
FROM `bigquery-public-data.github_repos.commits`,
     UNNEST(repo_name) AS repo_name
GROUP BY repo_name
ORDER BY commit_count DESC
LIMIT 10;
""",
        'penjelasan': 'Menemukan repositori GitHub dengan commit terbanyak'
    },
    'Query 3: Analisis Cuaca Chicago': {
        'dataset': 'bigquery-public-data.chicago_taxi_trips.taxi_trips',
        'sql': """
SELECT
  EXTRACT(HOUR FROM trip_start_timestamp) AS jam,
  COUNT(*) AS jumlah_perjalanan,
  AVG(fare) AS rata_rata_tarif,
  AVG(trip_miles) AS rata_rata_jarak_mil
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2022
  AND fare IS NOT NULL
  AND trip_miles > 0
GROUP BY jam
ORDER BY jam;
""",
        'penjelasan': 'Pola perjalanan taksi Chicago berdasarkan jam dalam sehari'
    },
    'Query 4: Window Function - Running Total': {
        'dataset': 'bigquery-public-data.samples.shakespeare',
        'sql': """
SELECT
  corpus,
  word_count,
  SUM(word_count) OVER (
    ORDER BY word_count DESC
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS running_total,
  RANK() OVER (ORDER BY word_count DESC) AS rank_by_words
FROM (
  SELECT corpus, SUM(word_count) AS word_count
  FROM `bigquery-public-data.samples.shakespeare`
  GROUP BY corpus
)
ORDER BY word_count DESC
LIMIT 10;
""",
        'penjelasan': 'Menggunakan Window Function untuk menghitung running total dan rank karya Shakespeare'
    }
}

print('╔══════════════════════════════════════════════════════════════╗')
print('║         CONTOH QUERY GOOGLE BIGQUERY PUBLIC DATASETS        ║')
print('╚══════════════════════════════════════════════════════════════╝')
print('URL: https://console.cloud.google.com/bigquery')
print()

for title, qdata in bigquery_queries.items():
    print(f'\n{'='*65}')
    print(f'📊 {title}')
    print(f'{'='*65}')
    print(f'📁 Dataset: {qdata["dataset"]}')
    print(f'💡 Tujuan : {qdata["penjelasan"]}')
    print(f'\n🔷 SQL Query:')
    print(qdata['sql'])

# Simulasi hasil query dengan data lokal
print('\n=== Simulasi Hasil Query 3 (Pola Taksi per Jam) ===')
import numpy as np
np.random.seed(42)
sim_hours = list(range(24))
sim_trips = [int(x) for x in np.random.poisson([500 if h in range(7,9) or h in range(17,20)
                                                  else 200 if h > 6 else 50 for h in sim_hours])]
sim_fare = [round(np.random.uniform(12, 25) + (2 if h in range(7,9) or h in range(17,20) else 0), 2)
            for h in sim_hours]

sim_result = pd.DataFrame({'jam': sim_hours, 'jumlah_perjalanan': sim_trips, 'rata_rata_tarif_usd': sim_fare})
print(sim_result.to_string(index=False))

## 6. Data Lake Simulation

Implementasi struktur folder Data Lake dengan tiga zona: **Bronze** (raw), **Silver** (processed), dan **Gold** (curated/aggregated).

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import tempfile
from pathlib import Path
from datetime import datetime, timedelta

# ============================================================
# Buat struktur Data Lake
# ============================================================
lake_root = Path(tempfile.mkdtemp()) / 'data_lake'

zones = {
    'bronze': lake_root / 'bronze' / 'transaksi' / '2024' / '01',
    'silver': lake_root / 'silver' / 'transaksi',
    'gold':   lake_root / 'gold'   / 'analytics',
    'meta':   lake_root / '_metadata',
}

for zone_path in zones.values():
    zone_path.mkdir(parents=True, exist_ok=True)

print('=== Struktur Data Lake Dibuat ===')
for zone, path in zones.items():
    print(f'  [{zone.upper():8s}] {path}')

# ============================================================
# BRONZE ZONE: Data mentah as-is
# ============================================================
print('\n--- Menulis ke BRONZE ZONE (raw data) ---')
np.random.seed(42)

# Simulasi data mentah dengan kualitas buruk
raw_data = pd.DataFrame({
    'TGL_TRANSAKSI': ['15/01/2024', '15/01/2024', '16-01-2024', '17/01/2024',  # format tidak konsisten
                      '18/01/2024', '18/01/2024', '19/01/2024', '20/01/2024'],
    'ID_PRODUK': ['P001', 'P002', 'P003', None, 'P005', 'P001', 'P002', 'P005'],  # ada null
    'HARGA': ['8500000', '250000', '150000', '75000', '450000', '8500000', '250000', '450000'],  # string
    'QTY': [1, 2, 3, 5, 1, 1, 3, 2],
    'KOTA_PEMBELI': ['Jakarta', 'BANDUNG', 'surabaya', 'Yogyakarta', 'Medan',  # casing tidak konsisten
                     'Jakarta', 'Bandung', 'Surabaya'],
    'DISKON': ['0%', '5%', '10%', '0%', '15%', '0%', '5%', '10%'],  # ada simbol %
})

bronze_path = zones['bronze'] / 'raw_transaksi_20240115.csv'
raw_data.to_csv(bronze_path, index=False)
print(f'✅ Bronze: {bronze_path.name} ({bronze_path.stat().st_size} bytes)')
print('   Preview data mentah (kualitas buruk):')
print(raw_data.head(4).to_string(index=False))

# Simpan metadata
meta = {
    'source': 'sistem_pos_restoran',
    'ingested_at': datetime.now().isoformat(),
    'rows': len(raw_data),
    'format': 'csv',
    'zone': 'bronze'
}
with open(zones['meta'] / 'transaksi_meta.json', 'w') as f:
    json.dump(meta, f, indent=2)

# ============================================================
# SILVER ZONE: Cleaning & Standardization
# ============================================================
print('\n--- Proses ke SILVER ZONE (cleaned data) ---')

silver_df = raw_data.copy()

# Cleaning steps
silver_df['ID_PRODUK'] = silver_df['ID_PRODUK'].fillna('UNKNOWN')
silver_df['HARGA'] = silver_df['HARGA'].astype(float)
silver_df['DISKON_PCT'] = silver_df['DISKON'].str.replace('%', '').astype(float)
silver_df['KOTA_PEMBELI'] = silver_df['KOTA_PEMBELI'].str.title()
silver_df['TGL_TRANSAKSI'] = pd.to_datetime(
    silver_df['TGL_TRANSAKSI'].str.replace('-', '/'), format='%d/%m/%Y'
)
silver_df['TOTAL_HARGA'] = silver_df['HARGA'] * silver_df['QTY'] * (1 - silver_df['DISKON_PCT'] / 100)
silver_df = silver_df.drop(columns=['DISKON'])

silver_path = zones['silver'] / 'transaksi_clean_202401.parquet'
silver_df.to_parquet(silver_path, index=False)
print(f'✅ Silver: {silver_path.name} ({silver_path.stat().st_size} bytes)')
print('   Preview data yang sudah dibersihkan:')
print(silver_df.head(4).to_string(index=False))

# ============================================================
# GOLD ZONE: Aggregated / BI-ready data
# ============================================================
print('\n--- Proses ke GOLD ZONE (aggregated/BI-ready) ---')

gold_df = silver_df.groupby('ID_PRODUK').agg(
    total_transaksi=('QTY', 'count'),
    total_qty=('QTY', 'sum'),
    total_revenue=('TOTAL_HARGA', 'sum'),
    avg_diskon=('DISKON_PCT', 'mean')
).reset_index().rename(columns={'ID_PRODUK': 'produk_id'})
gold_df['revenue_share_pct'] = (gold_df['total_revenue'] / gold_df['total_revenue'].sum() * 100).round(1)

gold_path = zones['gold'] / 'revenue_per_produk_202401.parquet'
gold_df.to_parquet(gold_path, index=False)
print(f'✅ Gold: {gold_path.name} ({gold_path.stat().st_size} bytes)')
print('   Tabel agregasi siap untuk BI/Dashboard:')
print(gold_df.to_string(index=False))

# Tampilkan struktur akhir
print('\n=== Struktur Lengkap Data Lake ===')
for path in sorted(lake_root.rglob('*')):
    if path.is_file():
        rel = path.relative_to(lake_root)
        size = path.stat().st_size
        print(f'  {str(rel):<60} ({size:>6} bytes)')

print('\n✅ Simulasi Data Lake selesai!')

## Tugas Praktikum

### Soal 1
Perluas simulasi Document Store (`DocumentStore`) dengan menambahkan:
- Method `find_by_range(field, min_val, max_val)` untuk mencari dokumen dengan nilai dalam rentang tertentu
- Gunakan method ini untuk mencari mahasiswa dengan IPK antara 3.7 dan 4.0
- Tambahkan minimal 5 dokumen mahasiswa baru dan tampilkan hasil query

### Soal 2
Tambahkan fungsionalitas pada `KeyValueStore`:
- Method `lpush(key, *values)` dan `lrange(key, start, end)` untuk mensimulasikan List Redis
- Simulasikan antrian (queue) tugas dengan operasi push dan pop
- Contoh: antrian email notifikasi yang akan dikirim

### Soal 3
Lakukan perbandingan format file yang lebih komprehensif:
- Buat dataset dengan 500.000 baris
- Simpan dalam format: CSV, Parquet (gzip), Parquet (snappy), dan JSON
- Bandingkan ukuran file dan waktu baca/tulis dalam tabel
- Manakah format terbaik untuk analitik dan mengapa?

### Soal 4
Perluas simulasi Data Lake:
- Tambahkan zona Bronze untuk data dari sumber kedua (misalnya data produk)
- Lakukan JOIN antara data transaksi (Silver) dan data produk di Silver Zone
- Buat tabel Gold baru: `revenue_per_kategori` dengan kolom kategori, total_revenue, jumlah_transaksi
- Simpan hasilnya di Gold Zone dalam format Parquet